In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch 
import torch.nn as nn
import torch.nn.functional as F 
import torchvision 
from torch.utils.data import Dataset,DataLoader


In [7]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

train_data = torchvision.datasets.MNIST(root="~/torch_datasets",train=True,transform = transform ,download=False)

test_data = torchvision.datasets.MNIST(root="~/torch_datasets",train=False,transform=transform,download=False)

In [8]:
train_loader = torch.utils.data.DataLoader(train_data,batch_size=128,shuffle=True)

test_loader = torch.utils.data.DataLoader(test_data,batch_size = 32,shuffle =False)


## Autoencoder

In [20]:
class autoencoder(nn.Module):
    def __init__(self,input_shape):
        super().__init__()
        self.encoder_hl = nn.Linear(input_shape,128)
        self.encoder_output = nn.Linear(128,128)
        
        self.decoder_hl = nn.Linear(128,128)
        self.decoder_output = nn.Linear(128,input_shape)
        
    def forward(self, x):
        x = self.encoder_hl(x)
        x = torch.relu(x)
        x = self.encoder_output(x)
        x = torch.relu(x)
        
        x = self.decoder_hl(x)
        x = torch.relu(x)
        x = self.decoder_output(x)
        x = torch.relu(x)
        
        return x
        
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')        

In [23]:
model = autoencoder(784).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
epoch = 20
loss_fn = nn.MSELoss()

In [ ]:
for e in range(epoch):
    loss = 0
    for x,y in train_loader:
        x  = x.view(-1,784).to(device)
        optimizer.zero_grad()
        
        output = model(x)
        
        train_loss = loss_fn(output,y)
        train_loss.backward()
        optimizer.step()
        loss += train_loss.item()
        
    loss = loss/len(train_loader)
    
    print(f'Epoch{e+1}/{epoch} loss = {loss}' )